#### Задача 1 (15 баллов).

Напишем программу - примитивного голосового помощника, задача которого - помогать пользователю изучать выбранный язык. Такая программа должна уметь следующие вещи:

- разговаривать с пользователем голосом (и распознавать его речь)
- знать команды "переведи" и "зачитай вслух". Обе команды заставляют программу предложить пользователю ввести текст, который потом переводится и/или зачитывается программой.
- знать команду "практика": программа предлагает пользователю небольшой текст, выбранный случайным образом из базы данных программы. Пользователь должен зачитать текст вслух, а программа должна сравнить распознанный голос пользователя с исходным текстом и сообщить, если не совпало.
- опционально можно добавить функцию для ручного пополнения базы данных.

Язык выбираете какой угодно (из поддерживаемых библиотеками, конечно).

Если напишете не в классах, минус пять баллов из пятнадцати!

In [3]:
import os
import re
import random
import string
import pandas as pd

import asyncio
from translate import Translator

from gtts import gTTS

from playsound import playsound

import speech_recognition as sr

In [28]:
os.remove(r"C:\WINDOWS\Temp\audio0.mp3")

In [29]:
class PortugueseBot:
    def __init__(self):
        self.database = pd.read_csv(r"portuguese_bot_database.txt", sep='|')
        self.to_ru_translator = Translator(from_lang='pt', to_lang="ru")
        self.to_pt_translator = Translator(from_lang='ru', to_lang="pt")
        self.r = sr.Recognizer()

    def my_tokenizer(self, text):
        pattern = r'\w+|[^\w\s]'
        matches = re.findall(pattern, text)
        return matches
    
    def listen(self):

        tts0 = gTTS('Что будем делать? Скажите ключевое слово - переведи, зачитай вслух, практика, стоп.', lang='ru')
        tts0.save(r"C:\WINDOWS\Temp\audio0.mp3")
        playsound(r"C:\WINDOWS\Temp\audio0.mp3")

        with sr.Microphone() as source:
            audio = self.r.listen(source, 3, 6)
        command = self.r.recognize_google(audio, language='ru')
        print(command)
        
        return command


    def translate(self):
        tts1 = gTTS('С какого языка на какой вы хотите перевести текст?', lang='ru')
        tts1.save(r"C:\WINDOWS\Temp\audio1.mp3")
        playsound(r"C:\WINDOWS\Temp\audio1.mp3")

        with sr.Microphone() as source:
            audio = self.r.listen(source, 3, 6)
        langs = self.r.recognize_google(audio, language='ru')
        print(langs)

        translated = ''
        if langs == 'с русского на португальский':
            tts2 = gTTS('Введите текст на русском.', lang='ru')
            tts2.save(r"C:\WINDOWS\Temp\audio2.mp3")
            playsound(r"C:\WINDOWS\Temp\audio2.mp3")

            text = input('Текст: ')

            translated = self.to_pt_translator.translate(text)
        else:           
            tts3 = gTTS('Введите текст на португальском.', lang='ru')
            tts3.save(r"C:\WINDOWS\Temp\audio3.mp3")
            playsound(r"C:\WINDOWS\Temp\audio3.mp3")

            text = input('Текст: ')

            translated = self.to_ru_translator.translate(text)

        print(translated)
    
    def read(self):
        tts4 = gTTS('Введите текст на португальском, и я его прочитаю!', lang='ru')
        tts4.save(r"C:\WINDOWS\Temp\audio4.mp3")
        playsound(r"C:\WINDOWS\Temp\audio4.mp3")

        text = input('Текст: ')

        tts5 = gTTS(text, lang='pt')
        tts5.save(r"C:\WINDOWS\Temp\audio5.mp3")
        playsound(r"C:\WINDOWS\Temp\audio5.mp3")

    def practice(self):
        tts6 = gTTS('Прочитайте вслух текст, который я выведу на экран. После прочтения я исправлю ваши ошибки.', lang='ru')
        tts6.save(r"C:\WINDOWS\Temp\audio6.mp3")
        playsound(r"C:\WINDOWS\Temp\audio6.mp3")

        text_from_base = self.database['text'].tolist()[random.randint(0,9)]
        print(text_from_base)

        with sr.Microphone() as source:
            audio = self.r.listen(source, 5, 40)
        text_to_check = self.r.recognize_google(audio, language='pt')
        print(text_to_check)

        # насколько я поняла, при распознании речи знаки препинания у нас, логично, не распознаются
        # тогда буду пытаться сравнивать списки токенов, предварительно удалив пунктуацию из списка токенов для предложения из базы
        # также привожу все к нижнему регистру
        tokenized_read = self.my_tokenizer(text_to_check)
        tokenized_read_final = [token.lower() for token in tokenized_read]
        tokenized_from_base = self.my_tokenizer(text_from_base)
        tokenized_from_base_without_punct = [token.lower() for token in tokenized_from_base if token not in string.punctuation]

        mistakes = set(tokenized_from_base_without_punct) - set(tokenized_read_final)
        if len(mistakes) == 0:
            print('Отлично! Ошибок нет!')
        else:
            print('Эти слова вы прочитали неверно:')
            for i in mistakes:
                print(i)



In [30]:
my_bot = PortugueseBot()

In [18]:
while True:
    command = my_bot.listen()
        
    if command:
        if "переведи" in command.lower():
            my_bot.translate()
        elif "зачитай вслух" in command.lower():
            my_bot.read()
        elif "практика" in command.lower():
            my_bot.practice()
        elif "стоп" in command.lower():
            break

Переведи
с русского на португальский
Ei! Vamos ao cinema hoje?
Переведи
с португальского на русский
Эй! Пойдем сегодня в кино?
Зачитай вслух
стоп


In [34]:
while True:
    command = my_bot.listen()
        
    if command:
        if "переведи" in command.lower():
            my_bot.translate()
        elif "зачитай вслух" in command.lower():
            my_bot.read()
        elif "практика" in command.lower():
            my_bot.practice()
        elif "стоп" in command.lower():
            break

практика
O treinador Bruno Lage frisou esta sexta-feira que o Benfica tem de manter a consistência e dinâmica ofensiva.
o treinador Bruno lager Frisson esta sexta-feira que é o Benfica tem de manter a consistência e dinâmica ofensiva
Эти слова вы прочитали неверно:
frisou
lage


UnknownValueError: 

In [ ]:
# я не умею, видимо, нормально произносить слово стоп))))) никак его не распознает, потому и ошибка
# хотя ячейкой выше справился